In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [2]:
max_length = 50
batch_size = 128
learning_rate = 0.01
num_classes = 2
num_layers = 4
num_units = 256
vocab_size = 3000
embedding_size = 256
dropout_in_rate = 0.1
dropout_out_rate = 0.2

# model

In [3]:
class RNN:
    
    def __init__(self):
        self._create_placeholder()
        self._create_model()
    
    def _create_placeholder(self):
        self.is_training = tf.placeholder(shape=(), dtype=tf.bool, name='is_training')
        self.inputs = tf.placeholder(shape=[None, max_length], dtype=tf.int32, name='inputs')
        self.inputs_length = tf.placeholder(shape=[None], dtype=tf.int32, name='inputs_length')
        self.target_classes = tf.placeholder(shape=[None, num_classes], dtype=tf.int32, name='target_classes')
    
    def _create_model(self):
        embedded_inputs = self._embedding(self.inputs)
        _, encoder_state = self._encode(embedded_inputs)
        encoder_state = tf.layers.dense(encoder_state, num_units, activation=tf.nn.relu, name='hidden_layer')
        self.outputs_layer = tf.layers.dense(encoder_state, num_classes, activation=tf.nn.softmax, name='outputs_layer')
    
    def _embedding(self, inputs):
        lookup_table = tf.get_variable('lookup_table', shape=[vocab_size, embedding_size], dtype=tf.float32)
        embedded_inputs = tf.nn.embedding_lookup(lookup_table, inputs)
        return embedded_inputs
    
    def _encode(self, embedded_inputs):
        outputs, final_state = self._bidirectional_cell(embedded_inputs, num_layers, num_units, dropout_in_rate, dropout_out_rate)
        return outputs, final_state
    
    def _bidirectional_cell(self, inputs, num_layers, num_units, dropout_in_rate, dropout_out_rate):
        cell_fw = self._gru(num_layers, num_units, dropout_in_rate, dropout_out_rate, name='cell_fw')
        cell_bw = self._gru(num_layers, num_units, dropout_in_rate, dropout_out_rate, name='cell_bw')
        (fw_outputs, bw_outputs), (fw_state, bw_state) = tf.nn.bidirectional_dynamic_rnn(
            cell_fw=cell_fw,
            cell_bw=cell_bw,
            inputs=inputs,
            sequence_length=self.inputs_length,
            dtype=tf.float32,
            scope='bidirectional_cells')
        outputs = tf.concat([fw_outputs, bw_outputs], axis=-1)
        final_state = tf.reduce_sum([fw_state, bw_state], axis=0)
        print(final_state.shape)
        final_state = tf.concat(tf.unstack(final_state, axis=0), axis=-1)
        print('final_state: ', final_state.shape)
        return outputs, final_state
    
    def _gru(self, num_layers: int, num_units: int, dropout_in_rate: float, dropout_out_rate: float, name: str):
        cells = []
        for l in range(num_layers):
            cell = tf.nn.rnn_cell.GRUCell(num_units, tf.nn.relu, kernel_initializer=tf.orthogonal_initializer, name=name)
            if l == 0:
                cell = tf.nn.rnn_cell.DropoutWrapper(cell, input_keep_prob=1-dropout_in_rate)
            if l == num_layers-1:
                cell = tf.nn.rnn_cell.DropoutWrapper(cell, output_keep_prob=1-dropout_out_rate)
            cells.append(cell)
        return tf.nn.rnn_cell.MultiRNNCell(cells)
    

In [4]:
rnn = RNN()

Instructions for updating:
seq_dim is deprecated, use seq_axis instead
Instructions for updating:
batch_dim is deprecated, use batch_axis instead
(4, ?, 256)
final_state:  (?, 1024)


In [6]:
inputs = [1,2,3,4,5] + [0] * 45
fd = {
    rnn.inputs: [inputs],
    rnn.inputs_length: [5],
}

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    outputs = sess.run(rnn.outputs_layer, feed_dict=fd)
    print(outputs)

[[0.49860844 0.50139153]]
